In [1]:
#!pip install dbdpy
#!pip install dbfread
#!pip install pandas
#!pip install bar_chart_race
#!pip install matplotlib pandas

In [2]:
import os
import pandas as pd
from dbfread import DBF

In [3]:
#Especificamos la ruta de los archivos DFB
CARPETA_PALTA = "D:/Proyectos/Ranking Palta/ranking-palta/Datos"

#creamos una lista en donde se van a almacenar los dataframes
dataframes = []

for archivo in os.listdir(CARPETA_PALTA):
    if archivo.endswith(".DBF"):
        # Leemos el archivo
        rutaArchivo = os.path.join(CARPETA_PALTA, archivo)        
        tabla = DBF(rutaArchivo, encoding="latin1")
        df = pd.DataFrame(iter(tabla))
        dataframes.append(df)

# Concatenamos todos los Dataframes en uno solo
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Filtramos los RUC: "No disponibles"
df_concatenado=df_concatenado[df_concatenado["NRO_DOCU"]!="No Disponi."]

# Convertimos la columna 'FECHA' a texto
df_concatenado['FECHA'] = df_concatenado['FECHA'].astype(str)

# Creamos la lista de posiciones que queremos cambiar
posiciones = [4, 6]

# Creamos una función para insertar cualquier letra en posiciones que deseamos
def insertar_letras(valor, letra, posiciones):
    for posicion in sorted(posiciones, reverse=True):
        valor = valor[:posicion] + letra + valor[posicion:]
    return valor

# Aplicamos la función a la columna del dataframe
df_concatenado['FECHA_MOD'] = df_concatenado['FECHA'].apply(lambda x: insertar_letras(x, "/", posiciones))
df_concatenado['FECHA_MOD_FER'] = pd.to_datetime(df_concatenado["FECHA_MOD"], format = "%Y/%m/%d")
df_concatenado['Año'] = df_concatenado['FECHA_MOD_FER'].dt.year

# Creamos una tabla dinámica de los datos que queremos
tabla_dinamica= pd.pivot_table(
    df_concatenado,
    values=["FOB_DOLPOL"],
    index= "NRO_DOCU",
    columns= "Año",
    aggfunc= {
        "FOB_DOLPOL": "sum"},
    fill_value=0

)
# Sumar los valores a lo largo de las columnas (años)
tabla_dinamica['Total'] = tabla_dinamica.sum(axis=1)

# Si quieres que la columna 'Total' esté al final
#tabla_dinamica = tabla_dinamica.reindex(columns=[*tabla_dinamica.columns[:-1], 'Total'])
tabla_dinamica = tabla_dinamica.sort_values(by='Total', ascending=False)

# Asegúrate de que df_tabla_dinamica esté en el formato correcto
#df_tabla_dinamica = tabla_dinamica.fillna(0)  # Rellenar valores NaN con 0

# Aplanar el MultiIndex
#df_tabla_dinamica.columns = [f'{col[1]}' for col in df_tabla_dinamica.columns]

#df_tabla_dinamica=df_tabla_dinamica.cumsum()

#df_tabla_dinamica=tabla_dinamica.reset_index()

#df_tabla_dinamica.drop('NRO_DOCU', axis=1, inplace=True)

# Pasamos la fecha a un index
#df_tabla_dinamica.set_index("FECHA_MOD_FER", inplace = True)

#print(df_tabla_dinamica.columns)
#print(df_tabla_dinamica.index)

top_200 = tabla_dinamica.head(200)
top_200.to_excel("top200.xlsx", index=True)
print(top_200)


            FOB_DOLPOL                                                \
Año               2001     2002     2003     2004     2005      2006   
NRO_DOCU                                                               
20340584237          0        0  7244799  7296744  7067466  19919356   
20325117835          0        0        0        0        0    644718   
20501923428    2360639  2769023  4060557  5049996  6399283   5509034   
20555757469          0        0        0        0        0         0   
20461642706          0        0        0        0        0         0   
...                ...      ...      ...      ...      ...       ...   
20557880730          0        0        0        0        0         0   
20601115850          0        0        0        0        0         0   
20600477791          0        0        0        0        0         0   
20600865421          0        0        0        0        0         0   
20603725931          0        0        0        0        0      